In [1]:
%pip install pandas
import pandas as pd
import ast


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
df = pd.read_csv('tripadvisor-barcelona-restaurants-100reviews-2025-02-13.csv')


In [5]:
def extract_last_location(loc_str):
    try:
        loc_list = ast.literal_eval(loc_str)
        return loc_list[-1] if loc_list else None
    except (ValueError, SyntaxError):
        return None

df['location'] = df['original_location'].apply(extract_last_location)

df['cuisine'] = df['cuisines'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else None)

def extract_certificate_of_excellence(award_str):
    if isinstance(award_str, str):
        index = award_str.find('Certificate of Excellence')
        if index != -1:  # Si encontramos 'Certificate of Excellence'
            return award_str[index:index+30]  # 'Certificate of Excellence' tiene 26 caracteres
    return None

df['last_certificate_excellence'] = df['awards'].apply(lambda x: extract_certificate_of_excellence(x) if isinstance(x, str) else None)

df['top_tag'] = df['top_tags'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else None)

columns_to_keep = [
    'price_level', 'vegan_options', 'gluten_free', 
    'open_days_per_week', 'avg_rating', 'total_reviews_count', 
    'food', 'service', 'atmosphere', 'excellent', 'location', 
    'awards', 'cuisine', 'top_tag'
]

df_filtered = df[columns_to_keep]

df_filtered.to_csv('./cleaned.csv', index=False)


In [8]:
cuisine_unique_values = df['cuisine'].unique()
award_unique_values = df['awards'].unique()
top_tag_unique_values = df['top_tag'].unique()

print(f"Valores únicos en 'cuisine': {len(cuisine_unique_values)} valores diferentes")
print(cuisine_unique_values)

print(f"\nValores únicos en 'awards': {len(award_unique_values)} valores diferentes")
print(award_unique_values)

print(f"\nValores únicos en 'top_tag': {len(top_tag_unique_values)} valores diferentes")
print(top_tag_unique_values)


Valores únicos en 'cuisine': 66 valores diferentes
['Steakhouse' 'Seafood' 'Mediterranean' 'Cafe' 'Spanish' 'Mexican'
 'Fast food' 'Italian' 'European' 'Barbecue' 'Japanese' 'Deli' 'American'
 'French' 'Pizza' 'International' 'Sushi' None 'Healthy' 'Bar' 'Chinese'
 'Lebanese' 'African' 'German' 'Polish' 'Chilean' 'Brew Pub' 'Latin'
 'British' 'Indian' 'Peruvian' 'Asian' 'Catalan' 'Thai' 'Russian'
 'Caribbean' 'Greek' 'Wine Bar' 'Brazilian' 'Philippine' 'Grill' 'Fusion'
 'Hawaiian' 'Contemporary' 'Argentinian' 'Venezuelan' 'Diner' 'Pub'
 'Middle Eastern' 'Arabic' 'Irish' 'Gastropub' 'Dutch' 'Moroccan'
 'Hungarian' 'Turkish' 'Swiss' 'Central American' 'Nepalese'
 'Southern-Italian' 'South American' 'Colombian' 'Cuban' 'Soups' 'Tibetan'
 'Korean']

Valores únicos en 'awards': 272 valores diferentes
['Certificate of Excellence 2016' nan
 "Travellers' Choice, Certificate of Excellence 2020, Certificate of Excellence 2018, Certificate of Excellence 2017, Certificate of Excellence 2016"
 "Tra

In [14]:
## Count nulls

def null_summary(df):
    null_counts = df.isnull().sum() 
    null_percentage = (null_counts / len(df)) * 100 
    null_data = pd.DataFrame({
        'Null Count': null_counts,
        'Null Percentage': null_percentage
    })
    return null_data

# Missing data for each column individually
null_summary_df = null_summary(df_filtered)
print(null_summary_df)

# Missing data in the whole data matrix
total_missing_percentage = df_filtered.isnull().sum().sum() / (df_filtered.size) * 100
print("Total missing data (%): ", total_missing_percentage)

                     Null Count  Null Percentage
price_level                   0         0.000000
vegan_options                 0         0.000000
gluten_free                   0         0.000000
open_days_per_week          444        11.102776
avg_rating                    0         0.000000
total_reviews_count           0         0.000000
food                          4         0.100025
service                       3         0.075019
atmosphere                 1157        28.932233
excellent                     0         0.000000
location                      0         0.000000
awards                     1014        25.356339
cuisine                     155         3.875969
top_tag                       0         0.000000
Total missing data (%):  4.960168613581967
